# Проект урока 1. «Анализ данных о поездках на такси в Нью-Йорке»

## Шаг 1

Описание колонок:
* pickup_dt – период с точностью до часа
* pickup_month – месяц
* borough – район Нью-Йорка, из которого был сделан заказ (5 районов + аэропорт)
* pickups – число поездок за период (час)
* hday – является ли день праздничным/выходным; Y — да,  N — нет
* spd – скорость ветра в милях в час
* vsb – видимость
* temp – температура в градусах Фаренгейта
* dewp – точка росы по Фаренгейту
* slp – давление
* pcp_01 – количество осадков за час
* pcp_06 – количество осадков за 6 часов
* pcp_24 – количество осадков за 24 часа
* sd – глубина снега в дюймах

Каждая строка в датасете — запись не об одной конкретной поездке, а о всех поездках, сделанных за определенный час из того или иного района Нью-Йорка, и о том, какие были показатели погоды в этот час. 

## Шаг 2

In [1]:
import pandas as pd

In [2]:
# Чтение данных 
taxi = pd.read_csv("1_lesson_taxi_nyc.csv")

In [3]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [7]:
taxi.shape

(29101, 14)

In [8]:
taxi.dtypes

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [9]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29101 entries, 0 to 29100
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pickup_dt     29101 non-null  object 
 1   pickup_month  29101 non-null  object 
 2   borough       26058 non-null  object 
 3   pickups       29101 non-null  int64  
 4   hday          29101 non-null  object 
 5   spd           29101 non-null  float64
 6   vsb           29101 non-null  float64
 7   temp          29101 non-null  float64
 8   dewp          29101 non-null  float64
 9   slp           29101 non-null  float64
 10  pcp 01        29101 non-null  float64
 11  pcp 06        29101 non-null  float64
 12  pcp 24        29101 non-null  float64
 13  sd            29101 non-null  float64
dtypes: float64(9), int64(1), object(4)
memory usage: 3.1+ MB


In [4]:
# Заменим пробел на знак нижнего подчеркивания, чтобы было удобно обращаться к столбцам, в которых имеется пробел.

taxi_name = {'pcp 01': 'pcp_01', 
             'pcp 06': 'pcp_06',
             'pcp 24': 'pcp_24'}
taxi = taxi.rename(columns = taxi_name)

In [5]:
taxi.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')

In [6]:
# Проверим, все ли районы встречаются в данных одинаково часто. 

taxi.borough.value_counts()

Bronx            4343
EWR              4343
Staten Island    4343
Brooklyn         4343
Queens           4343
Manhattan        4343
Name: borough, dtype: int64

In [7]:
# Общее количество поездок
taxi.pickups.sum()

14265773

In [8]:
# Определим район Нью-Йорка, из которого было совершено наибольшее количество поездок.

taxi.groupby('borough').pickups.sum().sort_values(ascending=False)

borough
Manhattan        10367841
Brooklyn          2321035
Queens            1343528
Bronx              220047
Staten Island        6957
EWR                   105
Name: pickups, dtype: int64

In [9]:
min_pickups = taxi.groupby('borough').pickups.sum().idxmin()

In [10]:
min_pickups

'EWR'

**Определим, из каких районов по праздникам и выходным дням в среднем поступает больше заказов, чем в обычные дни.** Такие расчеты помогут компании оптимизировать предложение машин такси и понять, в каких районах в выходные дни нужно дополнительно стимулировать водителей выходить на смену, чтобы машин хватало.

In [11]:
taxi.groupby(['borough', 'hday']).pickups.mean().to_frame()

pickups
borough       hday             
Bronx         N       50.771073
              Y       48.065868
Brooklyn      N      534.727969
              Y      527.011976
EWR           N        0.023467
              Y        0.041916
Manhattan     N     2401.302921
              Y     2035.928144
Queens        N      308.899904
              Y      320.730539
Staten Island N        1.606082
              Y        1.497006

In [12]:
# Определим число поездок в каждом районе по месяцам чтобы выявить закономерности и сезонность (если есть)
pickups_by_mon_bor = taxi.groupby(['borough', 'pickup_month'], as_index=False)\
    .pickups.sum()\
    .sort_values('pickups', ascending=False)

In [13]:
pickups_by_mon_bor.head()

,borough,pickup_month,pickups
21,Manhattan,Jun,1995388
23,Manhattan,May,1888800
19,Manhattan,Feb,1718571
22,Manhattan,Mar,1661261
18,Manhattan,Apr,1648278


## Итоги

В этом проекте были проанализированы данные о поездках на такси в Нью-Йорке, получены инсайты для бизнес-решений и потренированы следующие навыки работы с библиотекой pandas:

- Чтение данных в формате csv
- Определение размера датафрейма и типов данных в его столбцах
- Переименование колонок
- Группировка и агрегация данных
- Сортировка данных
- Получение индексов с минимальными и максимальными значениями
- Группировка по нескольким колонкам
- Сохранение результата в новый датафрейм